In [2]:
import carla
import random

# Connect to the client and retrieve the world object
client = carla.Client('localhost', 2000)
world = client.get_world()

In [ ]:
client.get_available_maps()

In [ ]:
blueprint_library = world.get_blueprint_library()
static_props = blueprint_library.filter ('static.prop.*')
for prop in static_props:
    print(prop.id)

In [5]:
client.start_recorder(r"C:\Users\Becker\Desktop\CARLA_0.9.15\CarlaUE4\recordings\udi_scenario_2.log")

"""
CARLA Simulation Script based on ODD parameters:

- **Operational Environment:**
  - Urban downtown area (e.g., CARLA Town03) with well-marked roads and a 4-way intersection.
  - Weather: Light rain with intermittent overcast skies, representing midday lighting conditions.
  - Road speed limits (e.g., ~50 km/h) and a construction zone (indicated by traffic cones and NPC construction workers).

- **Dynamic Actors:**
  - Ego vehicle: vehicle.toyota.prius (set to autopilot).
  - Emergency vehicle: vehicle.ford.ambulance (set to autopilot; in a real test, custom emergency logic could be added).
  - Several NPC vehicles, pedestrians (including unpredictable behavior), and a cyclist.
"""

import carla
import random
import time

def main():
    # Connect to the CARLA server.
    client = carla.Client('localhost', 2000)
    client.set_timeout(10.0)
    world = client.get_world()
    world = client.load_world('Town05_Opt')

    # Set weather parameters: light rain, overcast skies at midday.
    weather = carla.WeatherParameters(
        cloudiness=20.0,      # Overcast skies.
        precipitation=10.0,   # Light rain.
        sun_altitude_angle=50.0  # Midday sun altitude.
    )
    world.set_weather(weather)
    print("Weather set to light rain with overcast conditions.")

    # Get the blueprint library and the map spawn points.
    blueprint_library = world.get_blueprint_library()
    spawn_points = world.get_map().get_spawn_points()
    if not spawn_points:
        raise Exception("No spawn points found. Check your map or CARLA configuration.")

    # -------------------------
    # 1. Spawn Ego Vehicle (Toyota Prius)
    # -------------------------
    ego_bp = blueprint_library.find('vehicle.toyota.prius')
    ego_spawn_point = random.choice(spawn_points)
    ego_vehicle = world.spawn_actor(ego_bp, ego_spawn_point)
    ego_vehicle.set_autopilot(True)  # Using autopilot mode for demonstration.
    print(f"Spawned ego vehicle (Toyota Prius) at {ego_spawn_point.location}.")

    # -------------------------
    # 2. Spawn NPC Vehicles
    # -------------------------
    npc_vehicle_blueprints = [
        blueprint_library.find('vehicle.audi.a2'),
        blueprint_library.find('vehicle.nissan.micra'),
        blueprint_library.find('vehicle.audi.tt'),
        blueprint_library.find('vehicle.mercedes.coupe_2020'),
        blueprint_library.find('vehicle.bmw.grandtourer'),
        blueprint_library.find('vehicle.harley-davidson.low_rider'),
        blueprint_library.find('vehicle.micro.microlino'),
        blueprint_library.find('vehicle.carlamotors.firetruck'),
        blueprint_library.find('vehicle.carlamotors.carlacola'),
        blueprint_library.find('vehicle.ford.mustang'),
        blueprint_library.find('vehicle.chevrolet.impala'),
        blueprint_library.find('vehicle.lincoln.mkz_2020'),
        blueprint_library.find('vehicle.citroen.c3'),
        blueprint_library.find('vehicle.dodge.charger_police'),
        blueprint_library.find('vehicle.nissan.patrol'),
        blueprint_library.find('vehicle.jeep.wrangler_rubicon'),
        blueprint_library.find('vehicle.mini.cooper_s'),
        blueprint_library.find('vehicle.mercedes.coupe'),
        blueprint_library.find('vehicle.dodge.charger_2020'),
        blueprint_library.find('vehicle.ford.crown'),
        blueprint_library.find('vehicle.seat.leon'),
        blueprint_library.find('vehicle.yamaha.yzf')
    ]
    npc_vehicles = []
    # Spawn a few NPC vehicles at random spawn points.
    for sp in random.sample(spawn_points, min(5, len(spawn_points))):
        bp = random.choice(npc_vehicle_blueprints)
        npc_vehicle = world.try_spawn_actor(bp, sp)
        if npc_vehicle is not None:
            npc_vehicle.set_autopilot(True)
            npc_vehicles.append(npc_vehicle)
    print(f"Spawned {len(npc_vehicles)} NPC vehicles.")

    # -------------------------
    # 3. Spawn Emergency Vehicle (Ford Ambulance)
    # -------------------------
    emergency_bp = blueprint_library.find('vehicle.ford.ambulance')
    emergency_spawn_point = random.choice(spawn_points)
    emergency_vehicle = world.spawn_actor(emergency_bp, emergency_spawn_point)
    emergency_vehicle.set_autopilot(True)
    # Custom logic (e.g., overriding traffic rules) could be added here.
    print(f"Spawned emergency vehicle (Ford Ambulance) at {emergency_spawn_point.location}.")

    # -------------------------
    # 4. Spawn Pedestrians
    # -------------------------
    walker_blueprints = blueprint_library.filter('walker.pedestrian.*')
    walker_list = []
    walker_controller_list = []
    # Generate random spawn transforms for pedestrians.
    walker_spawn_points = []
    for i in range(10):  # Spawn 10 pedestrians.
        loc = world.get_random_location_from_navigation()
        if loc is not None:
            walker_spawn_points.append(carla.Transform(loc))

    # Spawn walkers and attach AI controllers.
    walker_controller_bp = blueprint_library.find('controller.ai.walker')
    for spawn_point in walker_spawn_points:
        walker_bp = random.choice(walker_blueprints)
        walker = world.try_spawn_actor(walker_bp, spawn_point)
        if walker is not None:
            walker_list.append(walker)
            controller = world.spawn_actor(walker_controller_bp, carla.Transform(), attach_to=walker)
            walker_controller_list.append(controller)
            controller.start()
            # Direct the pedestrian to a random destination (simulate unpredictable behavior).
            destination = world.get_random_location_from_navigation()
            if destination is not None:
                controller.go_to_location(destination)
            controller.set_max_speed(1 + random.random())  # Speed between 1 and 2 m/s.
    print(f"Spawned {len(walker_list)} pedestrians.")

    # -------------------------
    # 5. Spawn a Cyclist
    # -------------------------
    # Using a motorcycle blueprint as a proxy for a cyclist.
    cyclist_blueprints = blueprint_library.filter('vehicle.yamaha.yzf')
    cyclist_vehicle = None
    if cyclist_blueprints:
        cyclist_bp = random.choice(cyclist_blueprints)
        cyclist_spawn_point = random.choice(spawn_points)
        cyclist_vehicle = world.spawn_actor(cyclist_bp, cyclist_spawn_point)
        if cyclist_vehicle is not None:
            cyclist_vehicle.set_autopilot(True)
            print(f"Spawned cyclist (Yamaha YZF) at {cyclist_spawn_point.location}.")

    # -------------------------
    # 6. Create Construction Zone
    # -------------------------
    # Select a spawn point location to serve as the construction zone.
    construction_location = random.choice(spawn_points).location
    construction_zone_transforms = []
    # Create a row of 5 cones (simulate a lane closure).
    for i in range(5):
        offset = i * 2.0  # Adjust spacing as needed.
        transform = carla.Transform(carla.Location(
            x=construction_location.x + offset,
            y=construction_location.y,
            z=construction_location.z))
        construction_zone_transforms.append(transform)

    cone_blueprint = blueprint_library.find('static.prop.warningconstruction')
    cones = []
    for transform in construction_zone_transforms:
        cone = world.try_spawn_actor(cone_blueprint, transform)
        if cone is not None:
            cones.append(cone)
    print(f"Spawned {len(cones)} traffic cones for the construction zone.")

    # -------------------------
    # 7. Run the Simulation
    # -------------------------
    simulation_duration = 60  # seconds.
    print(f"Running simulation for {simulation_duration} seconds. Observe interactions near the construction zone and intersection events.")
    start_time = time.time()
    while time.time() - start_time < simulation_duration:
        world.tick()
        time.sleep(0.05)  # Delay to limit CPU usage.

    # -------------------------
    # 8. Clean-Up: Destroy Actors
    # -------------------------
    print("Destroying actors...")
    for actor in npc_vehicles:
        actor.destroy()
    emergency_vehicle.destroy()
    ego_vehicle.destroy()
    for walker in walker_list:
        walker.destroy()
    for controller in walker_controller_list:
        controller.destroy()
    for cone in cones:
        cone.destroy()
    if cyclist_vehicle is not None:
        cyclist_vehicle.destroy()
    print("All actors destroyed. Simulation complete.")

if __name__ == '__main__':
    try:
        main()
    except KeyboardInterrupt:
        print("Simulation cancelled by user.")


client.stop_recorder

Weather set to light rain with overcast conditions.
Spawned ego vehicle (Toyota Prius) at Location(x=24.707123, y=-51.034554, z=0.300000).
Spawned 5 NPC vehicles.
Spawned emergency vehicle (Ford Ambulance) at Location(x=-47.600029, y=-22.850307, z=0.450000).
Spawned 8 pedestrians.
Spawned cyclist (Yamaha YZF) at Location(x=190.040817, y=13.617901, z=0.300000).
Spawned 5 traffic cones for the construction zone.
Running simulation for 60 seconds. Observe interactions near the construction zone and intersection events.


: 

In [ ]:
client.stop_recorder

In [ ]:
world = client.load_world('Town03_Opt')
client.replay_file(r"C:\Users\Becker\Desktop\CARLA_0.9.15\CarlaUE4\recordings\udi_scenario.log", 0, 0,0)